In [1]:
%load_ext watermark
%watermark

2022-04-26T11:37:55+02:00

CPython 3.6.7
IPython 6.4.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-175-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


In [2]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

In [4]:
n_core = 10
N = 512
print(linspace(1,N+1-N//n_core,n_core,dtype='int'))
print(linspace(N//n_core,N,n_core,dtype='int'))

# integer          , parameter :: ia(ni)=[1,257,513,769]
# integer          , parameter :: ib(ni)=[256,512,768,1024]

[  1  52 103 154 205 257 308 359 410 462]
[ 51 102 153 204 255 307 358 409 460 512]


In [5]:
def load_file_GUI(dir_string):

    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename(initialdir = dir_string,
                                       multiple=True)
    return file_path

In [6]:
%run ../../GiantMol/Functions/1.2/data_fct_Adrien_point_by_point
matplotlib.rcParams.update({'font.size': 25})

In [7]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [8]:
def data_retrieve(all_subdir,points_and_coord, condition_parameters, slash_cfg,**kwargs):

    myslashpos = slash_cfg[0]
    slashcond = slash_cfg[1]

    # determining number of elements on each repetition
    try:
        num_runs = ['Try'+str(kwargs.get('forcenumtry'))]
        num_runs_aux = [runs[myslashpos[slashcond+1]+1:] for runs in all_subdir if list(points_and_coord.keys())[0] in runs]
        num_runs_aux = list(dict.fromkeys(num_runs_aux))
        num_runs_aux = len(num_runs_aux)
        all_subdir = [x for w,x in enumerate(all_subdir) if (w-kwargs.get('forcenumtry'))%num_runs_aux==0]
    except:
        num_runs = [runs[myslashpos[slashcond+1]+1:] for runs in all_subdir if list(points_and_coord.keys())[0] in runs]
        num_runs = list(dict.fromkeys(num_runs))

    # number of repetitions
    print('> Points |',len(points_and_coord))
    print('> Simulations pour chaque point |', num_runs)
    
    data0 = [[] for i in range(len(points_and_coord))] # file path to SimuType4
    data_address = [[] for i in range(len(points_and_coord))]

    # Variables à deux coordonnées : [point, try]
    shapevar = (len(points_and_coord),len(num_runs))
    Time        = []
    Temperature = []
    
    t0 = time.clock()
    print("Hello")

    # write variables
    for k, address in enumerate(all_subdir):

    # in-loop variables
        pnt = k // len(num_runs)  # actual point
        rep = k  % len(num_runs)          # actual repetition

        # get only .dat files in each simulation directory
        onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
        # build path file
        data0[pnt].append('{}/{}'.format(address,sort(onlyfiles)[0].strip('.dat')))
        data_address[pnt].append(address)

        # load fluorescence and T
        Time.append( loadtxt(str(data0[pnt][0])+'.dat',unpack=True)[0] )
        Temperature.append( loadtxt(str(data0[pnt][0])+'.dat',unpack=True)[4:7] )
        
        # load cloud size before injection
        if not(rep % len(num_runs)):
            print( "Point n°", pnt )
        
        print(f'{pnt:02}','-',f'{rep:02}',' > ',data0[pnt][rep])
        
        if k == len(points_and_coord)*len(num_runs) - 1:
            break

    t1 = time.clock() - t0
    print("Time elapsed: ", t1, 's') # CPU seconds elapsed (floating point)
    print("Time elapsed: ", t1/60, 'm') # CPU seconds elapsed (floating point)
    
    data_name = [data_address, data0]
    
    return data_name, num_runs, Time, Temperature 

# Extract data as npz

In [25]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint')
print(file_path)

('/home/adrian/RemoteFS/Rivendel/Simulations/20220421/DC11_RF00/Try00/Temp_SimuType0_N00512_Vrf0060_Udc0.7000D+01V_nt1000.dat',)


In [26]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Rivendel/Simulations/20220421/DC11_RF00/Try00
> Filename : Temp_SimuType0_N00512_Vrf0060_Udc0.7000D+01V_nt1000.dat
> myslashpos | [0, 5, 12, 21, 30, 42, 51, 61]
> slashcond | 6
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 19


In [27]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC11_RF00', 'DC11_RF01', 'DC11_RF02', 'DC11_RF03', 'DC11_RF04', 'DC11_RF05', 'DC11_RF06', 'DC11_RF07', 'DC11_RF08', 'DC11_RF11', 'DC11_RF12', 'DC11_RF13', 'DC11_RF14', 'DC11_RF15', 'DC11_RF16', 'DC11_RF17', 'DC11_RF18', 'DC11_RF22', 'DC11_RF23']
000 > DC11_RF00
001 > DC11_RF01
002 > DC11_RF02
003 > DC11_RF03
004 > DC11_RF04
005 > DC11_RF05
006 > DC11_RF06
007 > DC11_RF07
008 > DC11_RF08
009 > DC11_RF11
010 > DC11_RF12
011 > DC11_RF13
012 > DC11_RF14
013 > DC11_RF15
014 > DC11_RF16
015 > DC11_RF17
016 > DC11_RF18
017 > DC11_RF22
018 > DC11_RF23
> condition names ['DC', 'RF']
> number of points 19
> N_ions = 512
> e_GMol = 50


In [28]:
## Data loading (1D arrays)
data_name, num_runs, Time, Temperature \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power  ,forcenumtry=28

> Points | 19
> Simulations pour chaque point | ['Try00']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20220421/DC11_RF00/Try00/Temp_SimuType0_N00512_Vrf0060_Udc0.7000D+01V_nt1000
Point n° 1
01 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20220421/DC11_RF01/Try00/Temp_SimuType0_N00512_Vrf0061_Udc0.7000D+01V_nt1000
Point n° 2
02 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20220421/DC11_RF02/Try00/Temp_SimuType0_N00512_Vrf0062_Udc0.7000D+01V_nt1000
Point n° 3
03 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20220421/DC11_RF03/Try00/Temp_SimuType0_N00512_Vrf0063_Udc0.7000D+01V_nt1000
Point n° 4
04 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20220421/DC11_RF04/Try00/Temp_SimuType0_N00512_Vrf0064_Udc0.7000D+01V_nt1000
Point n° 5
05 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20220421/DC11_RF05/Try00/Temp_SimuType0_N00512_Vrf0065_Udc0.7000D+01V_nt1000
Point n° 6
06 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20220421/DC1

In [29]:
for j,k in enumerate(data_name[0]):
    cond = k[0][52:61]
    print(cond)
    outfile = 'Time_and_temp_RFHEAT_N0512_'+str(cond)
    np.savez( outfile, time=Time[j], temp = mean(Temperature[j],axis=0)) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)

DC11_RF00
DC11_RF01
DC11_RF02
DC11_RF03
DC11_RF04
DC11_RF05
DC11_RF06
DC11_RF07
DC11_RF08
DC11_RF11
DC11_RF12
DC11_RF13
DC11_RF14
DC11_RF15
DC11_RF16
DC11_RF17
DC11_RF18
DC11_RF22
DC11_RF23


In [31]:
# load check
cond = 'DC11_RF18'
outfile = 'Time_and_temp_RFHEAT_N0512_'+str(cond)
with load(outfile+'.npz') as data:
    a = data['time']
    b = data['temp']
print(a)
print(b)
print(shape(a))
print(shape(b))

[0.00100062 0.00100112 0.00100162 ... 0.01005262 0.01005312 0.01005362]
[1.92375278e-03 2.42508665e-03 2.52911492e-03 ... 9.08200632e+01
 7.94317444e+01 1.00054870e+02]
(18107,)
(18107,)


In [32]:
whos

Variable                                       Type                          Data/Info
--------------------------------------------------------------------------------------
C_e                                            float                         1.602e-19
N                                              int                           512
Omega                                          float                         12880529.879718151
Path                                           type                          <class 'pathlib.Path'>
Temperature                                    list                          n=19
Time                                           list                          n=19
a                                              ndarray                       18107: 18107 elems, type `float64`, 144856 bytes (141.4609375 kb)
all_subdir                                     list                          n=19
b                                              ndarray                   

# Work with data

In [32]:
file_path = array(load_file_GUI('/home/adrian/Documents/Programmes/Python'))
print(file_path)

for j,k in enumerate(file_path):
    cond = k[0][52:61]

['/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC11_RF00.npz'
 '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC11_RF01.npz'
 '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC11_RF02.npz'
 '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC11_RF03.npz'
 '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC11_RF04.npz'
 '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC11_RF05.npz'
 '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC11_RF06.npz'
 '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC11_RF07.npz'
 '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC11_RF08.npz'
 '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_tem

In [53]:
all_Urf = [60,61,62,63,64,65,66,67,68,
           00,00,
           62.2,62.4,62.6,62.8,63.2,63.4,63.6,63.8,40,45,50,55,70,75,80,85]
all_N = [1024,512]
Urf = []
file_path_tmp = []
for l,m in enumerate(all_N):
    N_str = str(m)
    print(N_str)
    Urf.append([])
    file_path_tmp.append([])
    for j,k in enumerate(file_path):
        if str(m) in k:            
#             print(str(m),k)
            str_pos = k.find(N_str)
            cond = k[str_pos+5:str_pos+14]
#             print(cond)
            str_pos = cond.find('RF')
            RF_cond = cond[str_pos+2:str_pos+4]        
#             print(cond,RF_cond)
            Urf[l].append(all_Urf[int(RF_cond)])
            file_path_tmp[l].append(k)
#             print('not found')
    print(sort(Urf[l]))

1024
[40.  45.  50.  55.  60.  61.  62.  62.2 62.4 62.6 62.8 63.  63.2 63.4
 63.6 63.8 64.  65.  66.  67.  68.  70.  75.  80.  85. ]
512
[55.  60.  61.  62.  62.2 62.4 62.6 62.8 63.  63.2 63.4 63.6 63.8 64.
 65.  66.  67.  68.  70. ]


In [76]:
# load check
# cond = 'DC11_RF00'
# outfile = 'Time_and_temp_RFHEAT_'+str(cond)
Time = []
Temperature = []
for l,m in enumerate(all_N):
    Time.append([])
    Temperature.append([])    
    for j,k in enumerate(file_path_tmp[l]):
        with load(k) as data:
            Time[l].append( data['time'] )
            Temperature[l].append( data['temp'] )
        print(j,k)
    print(shape(Time))
    print(shape(Temperature))

0 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC11_RF00.npz
1 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC11_RF01.npz
2 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC11_RF02.npz
3 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC11_RF03.npz
4 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC11_RF04.npz
5 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC11_RF05.npz
6 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC11_RF06.npz
7 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC11_RF07.npz
8 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC11_RF08.npz
9 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_

In [92]:
imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(10); clf();
j = 1
l = 11
print(j,l,Urf[j][l])
#     subplot(1,9,i+1)
semilogy(Time[j][l]*1e3,savgol_filter(Temperature[j][l],31,1),label=' Vrf = '+str(Urf[j][l]))
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
grid()
legend(fontsize=12)

imax 2
1 11 62.6


In [55]:
# Functions for fit

def func2(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+(t-t0)**2)**D + B

def func3(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+abs(t-t0))**D + B

def dfunc2_dt(t, t0, A, B,  C, D):
    return 10**(A*(C + (t - t0)**2)**(-D)*(t - t0) + B)*(-A*D*(C + (t - t0)**2)**(-D)*(t - t0)*(2*t - 2*t0)/(C + (t - t0)**2) + A*(C + (t - t0)**2)**(-D))*log(10)

In [213]:
# Fit all temperature curves with above function
# savgol_filter used to filter a little bit
# smooth the temperatures that oscillates a bit too much
imax = 26
T_max = 85
T_thr = 1
fit_param  = [] # zeros((len(all_N),imax,5))
fit_pcov   = [] # zeros((len(all_N),imax,5,5))
my_Urf     = []
my_order   = []
gone_wrong = []

figure(1).clf()
for l,m in enumerate(all_N):
    fit_param.append([])
    fit_pcov.append([])
    my_Urf.append([])
    my_order.append([])
    for j,k in enumerate(file_path_tmp[l]):
        print('<<<',j,'>>>')
        k_max = argmin(abs(Temperature[l][j] - T_max))
        k_thr = argmin(abs(Temperature[l][j] - T_thr))

        i_aux = where(diff(abs(diff(Temperature[l][j][k_thr:k_max])))>2)
        if size(i_aux) != 0 :
            if (i_aux[0][0]+k_thr < k_max):
                k_max = i_aux[0][0]+k_thr

        k_min = k_thr - int(2*(k_max - k_thr))
        if k_min < 0: k_min = 0
    #         print(k_min)

        x  = Time[l][j][k_min:k_max]
        print(Urf[l][j])
        print(len(Temperature[l][j]),k_max,k_thr,k_min)
        try:
            y  = savgol_filter(Temperature[l][j][k_min:k_max],31,1)
        except:
            y  = Temperature[l][j][k_min:k_max]
            print('no filter')

        x0 = Time[l][j][k_thr]

        A0  = 1
        B0  = log10(Temperature[l][j][k_thr])
        C0  = 1.0e-8
        D0  = 0.5
        p0  = array([x0, A0, B0,  C0, D0])

        my_order[l].append(j)

        try:
            popt, pcov = curve_fit(func2, x, log10(y), p0, maxfev= 500000)
            
            if popt[1] > 1e-5:
                fit_param[l].append([popt[0], popt[1], popt[2], popt[3], popt[4]])
                fit_pcov[l].append(pcov)
                my_Urf[l].append(Urf[l][j])
                        
                y_fit = func2(x,popt[0], popt[1], popt[2], popt[3], popt[4])
                y_fit = 10**y_fit

                subplot(8,8,j+1+(l*25))
                plot(x*1e3, y    ,label=' Vrf = '+str(Urf[l][j]))
                plot(x*1e3, y_fit,label='Fit',ls='--')
                grid()
                legend(fontsize=10)
    #             plt.gca().invert_xaxis()
    #             plt.gca().invert_yaxis()
                # select cases
                # non zero fit value
        except TypeError:
            print('TypeError for ',Urf[l][j])
#             gone_wrong.append(i)
tight_layout()

<<< 0 >>>
60
31892 31732 30753 28795
<<< 1 >>>
61
23609 21381 20630 19128
<<< 2 >>>
62
13529 12618 12171 11277
<<< 3 >>>
63
8329 5953 5630 4984
<<< 4 >>>


/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in power
  after removing the cwd from sys.path.


64
5901 5898 5300 4104
<<< 5 >>>
65
6127 4549 3861 2485
<<< 6 >>>
66
3489 3475 2948 1894
<<< 7 >>>
67
4980 2467 2130 1456
<<< 8 >>>
68
4388 1399 974 124
<<< 9 >>>
62.2
13609 11524 10515 8497
<<< 10 >>>
62.4
5485 3176 2419 905
<<< 11 >>>
62.6
13522 10221 9333 7557
<<< 12 >>>
62.8
11919 8457 7843 6615
<<< 13 >>>
63.2
10089 8804 7787 5753
<<< 14 >>>
63.4
9969 7276 6310 4378
<<< 15 >>>
63.6
10673 6787 6125 4801
<<< 16 >>>
63.8
9071 6734 6090 4802
<<< 17 >>>
40
100000 372 372 372
no filter
TypeError for  40
<<< 18 >>>
45
100000 850 850 850
no filter
TypeError for  45
<<< 19 >>>
50
500000 43 43 43
no filter
TypeError for  50
<<< 20 >>>
55
271474 270390 269529 267807
<<< 21 >>>
70
3493 1306 943 217
<<< 22 >>>
75
1185 33 0 0


/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


<<< 23 >>>
80
1903 9 6 0
no filter
<<< 24 >>>
85
882 1 0 0
no filter
TypeError for  85
<<< 0 >>>
60
73159 72307 71469 69793
<<< 1 >>>
61
48657 43243 42559 41191
<<< 2 >>>
62
38680 31068 30739 30081
<<< 3 >>>
63
8911 3110 2368 884
<<< 4 >>>
64
11607 11251 10623 9367
<<< 5 >>>
65
9057 6779 6401 5645
<<< 6 >>>
66
10146 5100 4427 3081
<<< 7 >>>
67
5333 3937 3182 1672
<<< 8 >>>
68
6220 915 273 0
<<< 9 >>>
62.2
25868 23344 22612 21148
<<< 10 >>>
62.4
2901 2342 1808 740
<<< 11 >>>
62.6
25602 21265 20745 19705
<<< 12 >>>
62.8
4267 1897 1332 202
<<< 13 >>>
63.2
22535 15325 14697 13441
<<< 14 >>>
63.4
16338 14206 13654 12550
<<< 15 >>>
63.6
17650 12061 11290 9748
<<< 16 >>>
63.8
18107 12886 12348 11272
<<< 17 >>>
55
500000 498729 498729 498729
no filter
TypeError for  55
<<< 18 >>>
70
3167 1543 1149 361


In [230]:
# select only Urf values
# for which non zero fit
# and sort ascending Urf
Y = Urf
X = my_order
Urf_order = [[],[]]
my_order_tmp  = [[],[]]
imax_order = []
for l,m in enumerate(all_N):    
    Urf_order[l] = [x for x,_ in sorted(zip(Y[l],X[l]))]
    my_order_tmp[l] = [x for _,x in sorted(zip(Y[l],X[l]))]
#     imax_order.append(len(Time[l]))
#     print('==')

# resort all variables
# in order so
# Urf ascending

fit_param_order = [] # zeros((len(all_N),max(imax_order),5))
fit_pcov_order  = [] # zeros((len(all_N),max(imax_order),5,5))
Time_order = []
Temperature_order = []
for l,m in enumerate(all_N):
    fit_param_order.append([])
    fit_pcov_order.append([])
    Time_order.append([])
    Temperature_order.append([])
    for i,j in enumerate(my_order_tmp[l]):
        fit_param_order[l] = fit_param[l][j]
        fit_pcov_order[l]  = fit_pcov[l][j]
        Time_order[l].append(Time[l][j])
        Temperature_order[l].append(Temperature[l][j])
        print(Urf[l][j])

40
45
50


IndexError: list index out of range

In [167]:
# draw one temp and fit
figure(3); clf()
cm = pylab.get_cmap('jet')

l = 1
j = 18

print(Urf_order[l][j])
# print(fit_pcov_order[i,:])
y_fit = func2(Time_order[l][j],
              fit_param_order[l,j,0],
              fit_param_order[l,j,1],
              fit_param_order[l,j,2],
              fit_param_order[l,j,3],
              fit_param_order[l,j,4])
y_fit = 10**y_fit
subplot(121)
plot((Time_order[l][j])*1e3,Temperature_order[l][j],color='r')
plot((Time_order[l][j])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
ylabel('T [K]')
grid()

# xlim(3.6,3.7)
# ylim(-1,10)
subplot(122)
semilogy((Time_order[l][j])*1e3,savgol_filter(Temperature_order[l][j],31,1),color='r')
semilogy((Time_order[l][j])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
grid()
tight_layout()

70


In [130]:
Temperature_order[l][j]

0.0025310848757068563

In [75]:
# draw fit parameters
# as a function of Urf
str_labels = ['t0', 'A', 'B', 'C', 'D']
figure('Param'); clf();
for i in range(0,5):
    subplot(1,5,i+1)
    if i == 2:
        plot(Urf_order,10**(fit_param_order[:,i]),'o-', label=str_labels[i])
    else:
        plot(Urf_order,(fit_param_order[:,i]),'o-', label=str_labels[i])
    xlabel('Urf')
    grid()
    legend()

In [65]:
# Draw Heating rate H
# as a function of temperature T
xa = -2
xb = 50
ya = 6e-6
yb = 1e0
figure(29,clear='True')
ax = subplot(111)
for i in [0,1,13,17]: #  range(0,imax_order)
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
#     ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6,label=str(Urf[i]),color=cm(i/imax_order))
    ax.semilogy(savgol_filter(Temperature_order[i],31,1),dy*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
#     ax.semilogy(Temperature_order[i][:-1],diff(Temperature_order[i])*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
    ax.set_xlabel('T [K]')
    ax.set_ylabel('Heating rate [K/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
ax.set_xlim([xa,xb])
ax.set_ylim([ya,yb])
        
# locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
# ax.yaxis.set_major_locator(locmaj)
# locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
# ax.yaxis.set_minor_locator(locmin)
# ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.legend(title=r'$U_{RF}$',ncol=2)
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

In [66]:
# Draw Heating rate H
# as a function of temperature T
xa = -2
xb = 50
ya = 6e-6
yb = 1e0
coucou = []
figure(31,clear='True')
ax = subplot(111)
for i in [0,1,13,17]: #  range(0,imax_order)
    t0, A, B, C, D = fit_param_order[i,:]
    x  = savgol_filter(Temperature_order[i],31,1)
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D) /x
#     ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6,label=str(Urf[i]),color=cm(i/imax_order))
    ax.semilogy(x,dy*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order),marker='+')
#     ax.semilogy(Temperature_order[i][:-1],diff(Temperature_order[i])*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
    coucou.append( argmin(abs(x-0.5)) )

#     ax.vlines(x[coucou],1e-5,0.5e0,color=cm(i/imax_order))
    ax.set_xlabel('T [K]')
    ax.set_ylabel('Heating rate / T [/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
ax.set_xlim([xa,xb])
ax.set_ylim([ya,yb])
        
# locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
# ax.yaxis.set_major_locator(locmaj)
# locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
# ax.yaxis.set_minor_locator(locmin)
# ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.legend(title=r'$U_{RF}$',ncol=2)
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

In [508]:
X = []
Y = []
for i,j in enumerate([0,1,13,17]):
#     x = savgol_filter(Temperature_order[i],31,1)
    x  = func2(Time_order[i],t0, A, B, C, D)
    X.append(Urf_order[j])
    Y.append(x[coucou[i]])
print(X,Y)
figure('test',clear='True')
plot(X,Y)

[55.0, 60.0, 65.0, 70.0] [3.816836351572443, 2.82087657947867, 2.031329563329401, -0.43716055296305467]


In [489]:
110/3493 * 15, x[15]

(0.47237331806470084, 0.00586806794642175)

# t0 and T as a function of Urf

In [103]:
t0    = zeros((len(all_N),imax_order))
y_t0  = zeros((len(all_N),imax_order))
dy_t0 = zeros((len(all_N),imax_order))
for l,m in enumerate(all_N):
    for i,j in enumerate(Urf_order[l]):
        t0[l,i], A, B, C, D = fit_param_order[l,i,:]
        y  = func2(t0[l,i],t0[l,i], A, B, C, D);
        y_t0[l,i]  = 10**y
        dy_t0[l,i] = dfunc2_dt(t0[l,i], t0[l,i], A, B,  C, D)

/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [105]:
def my_lin(x,a,b):
    return a*x+b
def my_power_law(x,a,b):
    return a*(x**-b)

In [118]:
Urf_order[0],t0[0]*1e3

([40,
  45,
  50,
  55,
  60,
  61,
  62,
  62.2,
  62.4,
  62.6,
  62.8,
  63,
  63.2,
  63.4,
  63.6,
  63.8,
  64,
  65,
  66,
  67,
  68,
  70,
  85],
 array([11.31542034,  7.08538976,  6.25818546,  2.20963856,  2.92928387,
         2.47258066,  2.06312651,  3.81413069,  4.89359772,  4.15467384,
         1.48552736,  4.06182884,  4.04450634,  3.64891376, 16.37744656,
         0.        ,  0.        ,  5.66667292,  4.92087739,  1.46905812,
         0.98799315,  1.00462449,  0.        ,  0.        ,  0.        ]))

In [126]:
# log plot
figure(321,clear='True')
ax = subplot(111)
ax.loglog(Urf_order[1],t0[1][:-6]*1e3,ls='-',marker='+',mew=5,ms=10)
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'
# ax.axis([5*10, 72, 1, 200])
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')

tight_layout()

In [84]:
figure(322); clf();
ax = subplot(111)
ax.plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
ax.grid()
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

popt, pcov = curve_fit(my_lin,Urf_order[2:-2],y_t0[2:-2])
ax.plot(Urf_order[2:-2],my_lin(array(Urf_order[2:-2]),popt[0],popt[1]),
    color='k',ls='--')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=5))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))

ax.text(64,0.815,f'$T(t_0) = {popt[0]:.2e} U_{{rf}} + {popt[1]:.2f}$',
            color='xkcd:black',fontsize='20',weight="bold",
            ha='left',va='center')

tight_layout()

# identique à 10**B

In [85]:
figure(323); clf();
subplot(111)
plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
grid()
xlabel('$U_{rf}$ [V]')
ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

tight_layout()

# identique à 10**B

In [ ]:
# check for frequencies